In [1]:
import numpy as np
import pandas as pd
import gensim
import re
import nltk
import csv
import gensim
import math
import timeit
import datetime
import operator
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("dark")

from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from pylab import plot, show, subplot, specgram, imshow, savefig
from gensim import corpora, models, similarities
import category_encoders as ce

In [2]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'

train_src = src + 'df_train_spacylemmat_fullclean.csv'
test_src = src + 'df_test_spacylemmat_fullclean.csv'

train_orig =  pd.read_csv(train_src)
test_orig =  pd.read_csv(test_src)

In [3]:
q1 = train_orig[['qid1', 'question1']]
q2 = train_orig[['qid2', 'question2']]

q2.rename(columns = {'question2':'question1'},inplace=True)
q2.rename(columns = {'qid2':'qid1'},inplace=True)

qfull = pd.concat([q1, q2])

test_questions = pd.DataFrame()
test_questions['question1'] = (test_orig.question1.tolist() + test_orig.question2.tolist())
test_questions['id'] = 0

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [ ]:
qdict = pd.Series(qfull.qid1.values,index = qfull.question1.values).to_dict()

In [ ]:
qdf = pd.DataFrame(qfull.question1.values, index = qfull.qid1.values)

In [ ]:
test_questions = test_questions.iloc[0:10000, :]

max_id = np.max(qdf.index.values) + 1
for i in range(len(test_questions)):
    if (test_questions['question1'][i]) in qdict:
        test_questions['id'] = qdict[test_questions['question1'][i]]
    else:
        test_questions['id'][i] = max_id
        qdict[test_questions['question1'][i]] = max_id
        max_id += 1
        #print(max_id)

In [4]:
df_id1 = train_orig[["qid1", "question1"]].drop_duplicates(keep="first").copy().reset_index(drop=True)
df_id2 = train_orig[["qid2", "question2"]].drop_duplicates(keep="first").copy().reset_index(drop=True)

df_id1.columns = ["qid", "question"]
df_id2.columns = ["qid", "question"]

print(df_id1.shape, df_id2.shape)

df_id = pd.concat([df_id1, df_id2]).drop_duplicates(keep="first").reset_index(drop=True)
print(df_id1.shape, df_id2.shape, df_id.shape)

import csv
dict_questions = df_id.set_index('question').to_dict()
dict_questions = dict_questions["qid"]

new_id = 538000 # df_id["qid"].max() ==> 537933

def get_id(question):
    global dict_questions 
    global new_id 
    
    if question in dict_questions:
        return dict_questions[question]
    else:
        new_id += 1
        dict_questions[question] = new_id
        return new_id
    
rows = []
max_lines = 10
if True:
    with open(src + 'df_test_spacylemmat_fullclean.csv', 'r', encoding="utf8") as infile:
        reader = csv.reader(infile, delimiter=",")
        header = next(reader)
        header.append('qid1')
        header.append('qid2')
        
        if True:
            print(header)
            pos, max_lines = 0, 10*1000*1000
            for row in reader:
                # "test_id","question1","question2"
                question1 = row[-3]
                question2 = row[-2]

                qid1 = get_id(question1)
                qid2 = get_id(question2)
                row.append(qid1)
                row.append(qid2)

                pos += 1
                rows.append(row)

(290654, 2) (299364, 2)
(290654, 2) (299364, 2) (537933, 2)
['id', 'is_duplicate', 'qid1', 'qid2', 'question1', 'question2', 'test_id', 'qid1', 'qid2']


In [5]:
dft = pd.DataFrame(rows)

In [6]:
testids = dft[[7,8]]
testids.columns = ['qid1', 'qid2']
testids.to_csv('test_qids.csv', index = False)